<a href="https://colab.research.google.com/github/CanopySimulations/canopy-python-examples/blob/master/loading_scalar_results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Upgrade Runtime
This cell ensures the runtime supports `asyncio` async/await, and is needed on Google Colab. If the runtime is upgraded, you will be prompted to restart it, which you should do before continuing execution.

In [0]:
!pip install "ipython>=7"

# Set Up Environment

### Import required libraries

In [0]:
!pip install -q 'canopy==6.1'

In [0]:
import canopy
import logging
import pprint

logging.basicConfig(level=logging.INFO)
pp = pprint.PrettyPrinter()

### Authenticate

In [0]:
authentication_data = canopy.prompt_for_authentication()

def create_session():
    return canopy.Session(authentication_data)

# Set Up Example

Run a quick ApexSim exploration.

In [3]:
async with create_session() as session:
    logging.info('Loading configs...')

    default_car = await canopy.load_default_config(
        session, 'car', 'Canopy F1 Car 2019')
    
    default_weather = await canopy.load_default_config(
        session, 'weather', '25 deg, dry')
    
    default_exploration = await canopy.load_default_config(
        session, 'exploration', 'Automated Test Monte Carlo')

    # We'll run a very small exploration for this example
    default_exploration.data.design.numberOfPoints = 3

    logging.info('Creating study...')

    study_id = await canopy.create_study(
        session,
        'apexSim',
        'Loading Scalar Results Example',
        [
            default_car,
            default_weather,
            default_exploration,
        ])
    
    logging.info('Waiting for study to complete...')

    wait_result = await canopy.wait_for_study(
        session,
        study_id,
        timeout_seconds=300)
    
logging.info('Sim Version: {}'.format(wait_result.document.sim_version))    
logging.info('Succeded Job Count: {}/{}'.format(
    wait_result.data.succeeded_job_count, 
    wait_result.data.job_count))

INFO:root:Loading configs...
INFO:root:Creating study...
INFO:root:Waiting for study to complete...
INFO:root:Sim Version: 1.3037
INFO:root:Succeded Job Count: 4/4


# Example: Loading Scalar Results

## Loading for a single job
We can load the scalar results for an individual job using the `include_scalar_results` parameter. Note we are specifically loading the `ApexSim` scalar results here. If we also wanted the `StraightSim` results we can fetch them in a second call.

In [5]:
async with create_session() as session:
    job_0 = await canopy.load_study_job(
        session,
        study_id,
        'ApexSim',
        job_index=0,
        include_scalar_results=True)
    
job_0.scalar_data

INFO:canopy.load_study_job:Loading job index 0


{'rLatBalF100': 0.5931529999999999,
 'rLatBalF200': 0.651191,
 'rLatBalF300': 0.689567,
 'kRollCar100': 374300.293621,
 'kRollCar200': 386790.823119,
 'kRollCar300': 389037.518536,
 'rAeroBal100': 0.5019060000000001,
 'rAeroBal200': 0.489875,
 'rAeroBal300': 0.49314399999999997,
 'rAeroBalvMax': 0.512822,
 'rhoAir': 1.182251,
 'hRideF100': 0.026491000000000004,
 'hRideF200': 0.015169,
 'hRideF300': -0.000541,
 'hRideR100': 0.133535,
 'hRideR200': 0.115568,
 'hRideR300': 0.08682899999999999,
 'zCarBrakingvMax': -0.296115,
 'aPitchBrakingvMax': -0.030741,
 'aPitchBrakingvMin': -0.027722000000000004,
 'aRockerFLBrakingvMax': -0.052287,
 'aRockerFLAcceleratingvMin': 0.005882,
 'aRockerRLBrakingvMax': -0.208146,
 'aRockerRLBrakingMax': 0.02084,
 'aRockerRLAcceleratingvMin': -0.027770999999999997}

We can request individual scalar results in our desired units:

In [6]:
job_0.scalar_as('hRideF200', 'mm')

15.169

You can also use the `scalar_as` function and omit the units, which will then use the preferred units saved in your Canopy account:

In [7]:
job_0.scalar_as('hRideF200')

0.015169

## Loading for an entire study

We can load the scalar results for the entire study using the `include_study_scalar_results` parameter. 

This loads the aggregated scalar results produced by the post-processing job when the study was run, so it is more efficient than loading the scalar results for each study individually.

If you did want to still load them individually, you could use the `include_job_scalar_results` parameter instead.

Note the aggregated scalar results contains the scalar results for every simulation run in the study (in this case both ApexSim and StraightSim).

In [0]:
async with create_session() as session:
    study = await canopy.load_study(
        session,
        study_id,
        include_study_scalar_results=True)

In [16]:
study.scalar_results.results_metadata

,name,units,description,simType
fullName,,,,
aPitchBrakingvMax:ApexSim,aPitchBrakingvMax,rad,aPitch under straight line braking at vMax.,ApexSim
aPitchBrakingvMin:ApexSim,aPitchBrakingvMin,rad,aPitch under straight line acceleration at vMin.,ApexSim
aRockerFLAcceleratingvMin:ApexSim,aRockerFLAcceleratingvMin,rad,aRockerFL under straight line acceleration at ...,ApexSim
aRockerFLBrakingvMax:ApexSim,aRockerFLBrakingvMax,rad,aRockerFL under straight line braking at vMax.,ApexSim
aRockerRLAcceleratingvMin:ApexSim,aRockerRLAcceleratingvMin,rad,aRockerRL under straight line acceleration at ...,ApexSim
...,...,...,...,...
vCarMaxElectricDRS:StraightSim,vCarMaxElectricDRS,m/s,Maximum vCar achievable on a flat track at wid...,StraightSim
zCarMax:StraightSim,zCarMax,m,Maximum zCar achieved during this simulation.,StraightSim
zCarMin:StraightSim,zCarMin,m,Minimum zCar achieved during this simulation.,StraightSim


In [17]:
study.scalar_results.results

,jobIndex,aPitchBrakingvMax:ApexSim,aPitchBrakingvMin:ApexSim,aRockerFLAcceleratingvMin:ApexSim,aRockerFLBrakingvMax:ApexSim,aRockerRLAcceleratingvMin:ApexSim,aRockerRLBrakingMax:ApexSim,aRockerRLBrakingvMax:ApexSim,hRideF100:ApexSim,hRideF200:ApexSim,hRideF300:ApexSim,hRideR100:ApexSim,hRideR200:ApexSim,hRideR300:ApexSim,kRollCar100:ApexSim,kRollCar200:ApexSim,kRollCar300:ApexSim,rAeroBal100:ApexSim,rAeroBal200:ApexSim,rAeroBal300:ApexSim,rAeroBalvMax:ApexSim,rLatBalF100:ApexSim,rLatBalF200:ApexSim,rLatBalF300:ApexSim,rhoAir:ApexSim,zCarBrakingvMax:ApexSim,tApexSim:Debug,tApexSimScalarResults:Debug,tApexSimVectorResults:Debug,tComponentSweepsVectorResults:Debug,tStraightSim:Debug,tStraightSimScalarResults:Debug,tStraightSimVectorResults:Debug,aCamberTyreF100:StraightSim,aCamberTyreF200:StraightSim,aCamberTyreF300:StraightSim,aCamberTyreFvMax:StraightSim,aCamberTyreR100:StraightSim,aCamberTyreR200:StraightSim,aCamberTyreR300:StraightSim,...,hRideUndertrayFrontvMax:StraightSim,hRideUndertrayMid100:StraightSim,hRideUndertrayMid200:StraightSim,hRideUndertrayMid300:StraightSim,hRideUndertrayMidvMax:StraightSim,hRideUndertrayRear100:StraightSim,hRideUndertrayRear200:StraightSim,hRideUndertrayRear300:StraightSim,hRideUndertrayRearvMax:StraightSim,kRollCar100:StraightSim,kRollCar200:StraightSim,kRollCar300:StraightSim,kRollCarvMax:StraightSim,lPushrodShimFL:StraightSim,lPushrodShimFR:StraightSim,lPushrodShimRL:StraightSim,lPushrodShimRR:StraightSim,lSpringShimFL:StraightSim,lSpringShimFR:StraightSim,lSpringShimRL:StraightSim,lSpringShimRR:StraightSim,rAeroBal100:StraightSim,rAeroBal200:StraightSim,rAeroBal300:StraightSim,rAeroBalvMax:StraightSim,rSlipGripMaxF_2kn:StraightSim,rSlipGripMaxR_2kn:StraightSim,rStiffnessBalF100:StraightSim,rStiffnessBalF200:StraightSim,rStiffnessBalF300:StraightSim,rStiffnessBalFvMax:StraightSim,rhoAir:StraightSim,vCarMax:StraightSim,vCarMaxDRS:StraightSim,vCarMaxElectric:StraightSim,vCarMaxElectricDRS:StraightSim,zCarMax:StraightSim,zCarMin:StraightSim,zHubOffsetF:StraightSim,zHubOffsetR:StraightSim
0,0,-0.030741,-0.027722,0.005882,-0.052287,-0.027771,0.020840,-0.208146,0.026491,0.015169,-0.000541,0.133535,0.115568,0.086829,374300.293621,386790.823119,389037.518536,0.501906,0.489875,0.493144,0.512822,0.593153,0.651191,0.689567,1.182251,-0.296115,6.536414,0.009102,0.517728,0.301744,1.152932,0.009713,0.534033,-0.051809,-0.050343,-0.049914,-0.049889,-0.010763,-0.011905,-0.014037,...,0.007730,0.085177,0.068894,0.045929,0.042301,0.131842,0.111540,0.081242,0.077532,378685.968330,384134.515592,391898.839422,392655.528223,0.001370,0.001370,-0.004044,-0.004044,0.0,0.0,0.0,0.0,0.496588,0.478288,0.476375,0.485354,0.118367,0.126531,0.682568,0.699690,0.730861,0.734433,1.182251,79.092254,84.233058,83.881193,88.927500,-0.285855,-0.339510,0.253510,0.261569
1,1,-0.034024,-0.030090,0.005631,-0.051499,-0.027292,0.020217,-0.180005,0.020449,0.008567,-0.008722,0.136174,0.119246,0.094933,376497.261477,389768.056360,394700.959955,0.518373,0.513907,0.536119,0.548199,0.592091,0.650931,0.685575,1.182251,-0.299097,6.325673,0.009827,0.617662,0.377539,1.150443,0.011710,0.628323,-0.051833,-0.050450,-0.050097,-0.050083,-0.010740,-0.011757,-0.013692,...,0.001998,0.083963,0.067951,0.045625,0.042570,0.134460,0.115111,0.086899,0.083917,380711.652420,387687.062007,395836.447571,396285.824002,-0.000963,-0.000963,-0.004864,-0.004864,0.0,0.0,0.0,0.0,0.511818,0.500620,0.511235,0.521479,0.118367,0.126531,0.682140,0.699541,0.728708,0.731689,1.182251,78.678201,83.537308,83.372004,88.014501,-0.286828,-0.338272,0.247539,0.263887
2,2,-0.031668,-0.028192,0.005864,-0.052621,-0.027413,0.020714,-0.204971,0.027424,0.016019,0.000185,0.136235,0.118482,0.089468,374954.448010,387696.539283,390066.318899,0.507077,0.493312,0.493568,0.512347,0.591632,0.649321,0.687264,1.182251,-0.297823,6.455958,0.009809,0.547089,0.296113,1.121402,0.009141,0.557586,-0.051800,-0.050315,-0.049884,-0.049860,-0.010736,-0.011790,-0.013826,...,0.008770,0.087088,0.070879,0.047697,0.0440

You also have access to the study input data as defined by the exploration:

In [18]:
study.scalar_results.inputs_metadata

,units,description,fullName,shortName,modifier
inputName,,,,,
car.chassis.carRunningMass.mCar+,kg,,car.chassis.carRunningMass.mCar,mCar,+
car.chassis.ICar+,kgm2,,car.chassis.ICar,ICar,+
car.chassis.hRideFSetup+,m,,car.chassis.hRideFSetup,hRideFSetup,+
car.chassis.hRideRSetup+,m,,car.chassis.hRideRSetup,hRideRSetup,+


In [19]:
study.scalar_results.inputs

,car.chassis.carRunningMass.mCar+,car.chassis.ICar+,car.chassis.hRideFSetup+,car.chassis.hRideRSetup+
0,7.148160,"[5.72223973726,0.0,0.0,0.0,11.44447947452,0.0,...",0.007905,0.012891
1,-8.617120,"[-17.92568042312,0.0,0.0,0.0,-35.85136084624,0...",0.002101,0.015405
2,2.523353,"[-1.21497055106,0.0,0.0,0.0,-2.42994110212,0.0...",0.008856,0.015461


We can convert this data to our desired units using the `session.units` helper:

In [21]:
async with create_session() as session:
    hRideF200_m = study.scalar_results.results['hRideF200:ApexSim']
    hRideF200_mm = session.units.convert_series_from_si(hRideF200_m, 'mm')

hRideF200_mm

0    15.169
1     8.567
2    16.019
Name: hRideF200:ApexSim, dtype: float64